# Task 2: Data Profiling, Cleaning & EDA
**Objective:** Profile, clean, and explore the solar dataset for Benin so it’s ready for comparison and region-ranking tasks.

This notebook includes:
- Summary statistics and missing-value report
- Outlier detection and cleaning
- Time series analysis
- Correlation and scatter plots
- Wind and temperature analysis
- Bubble charts

## Importing the dependencies

In [ ]:
# import dependencies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

## Setting the plotting style and Loading the Data

In [ ]:
# Set both plotting and display settings
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

# Load dataset
df = pd.read_csv('data/benin-malanville.csv')  # Adjust filename as needed
print(f"Dataset shape: {df.shape}")
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Quick overview
df.head()
df.info()





# Quick overview

df.head()
df.info()